# Tweaking learning rate on progressive resizing

When moving onto the next size, use the smaller learning rate.

TODO:

- Use bigger images for training.
- Try using different crop/squish after different batches.
- Why does it take such a long time to go onto the next epoch? It seems like it takes about 10 seconds. Maybe because I was using too big of a batch size?   -> Restarting the instance fixed this issue..
- 

Interesting find:

- When training on bigger size, the model struggled on the second epoch. This also happened to Jeremy as well. I thought it happened because I increased the image too much. But it happened even after lowering the gap. May have to use smaller learning rate.

In [7]:
from fastai.vision.all import *
from fastcore.parallel import *
path = Path()
trn_path = path/'train_images'

In [8]:
df = pd.read_csv('train.csv')
df.head()

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45


Turning DataLoaders into DataBlocks. We need to return 2 outputs, variety and disease.

In [9]:
arch = resnet18

In [10]:
img2variety = { r.image_id:r.variety for _,r in df.iterrows() }

In [11]:
def get_variety(p):
    "Turn path into image name"
    return img2variety[p.name]

## Customizing head

Instead of creating a new head, we can just tell our neural net to output 20 numbers instead of 10, first 10 for disease and last 10 for the variety. The neural net will figure out how to minimize the loss by itself.

In [12]:
orig_lf = CrossEntropyLossFlat()

def disease_err(inp,disease,variety): return error_rate(inp[:,:10],disease)
def variety_err(inp,disease,variety): return error_rate(inp[:,10:],variety)
def disease_loss(inp,disease,variety): return orig_lf(inp[:,:10],disease)
def variety_loss(inp,disease,variety): return orig_lf(inp[:,10:],variety)
def loss(pred,disease,variety): return orig_lf(pred[:,:10],disease)+orig_lf(pred[:,10:],variety)
err_metrics = (disease_err,variety_err)
all_metrics = err_metrics+(disease_loss,variety_loss)

In [13]:
def get_dls(img_sizes):
    """Return the dataloaders"""
    for size in img_sizes:
        print('*' * 40)
        print(f'Size: {size}'.center(40, '*'))
        print('*' * 40)
        dblock = DataBlock(
            blocks=(ImageBlock, CategoryBlock, CategoryBlock),
            n_inp=1,                                             
            get_items=get_image_files,
            get_y=[parent_label, get_variety],
            splitter=RandomSplitter(seed=42),
            item_tfms=Resize(224, method='squish'),                
            batch_tfms=aug_transforms(size=size),
        )
        dls = dblock.dataloaders(trn_path, bs=64)
        yield dls

In [15]:
def prog_sizing(img_sizes, epochs=1):
    """Progressively use bigger images for training."""
    dls_gen = get_dls(img_sizes)
    dls = next(dls_gen)
    learn = vision_learner(dls, arch, loss_func=loss, metrics=all_metrics, n_out=20).to_fp16()
    lr = 0.01
    learn.fine_tune(epochs, lr)
    for dls in dls_gen:
        # Half the learning rate when updating the image size.
        lr /= 2
        learn.dls = dls
        learn.fine_tune(epochs, lr)

After 20 epochs, it is better than training normally.

What would happen if we add progressive resizing?

## Using batch size 64:

### Using 10 epochs

Using 10 epochs doesn't seem to improve our score at all. It's just same as not using progressive sizing or multitask.

In [16]:
prog_sizing([32, 64, 128, 224], epochs=10)

****************************************
****************Size: 32****************
****************************************


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,4.242627,3.121841,0.687650,0.339741,2.023082,1.098759,00:29


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,2.754838,1.965791,0.460836,0.176358,1.389152,0.576639,00:21
1,2.078140,1.514551,0.367612,0.142239,1.082159,0.432392,00:25
2,1.723304,1.248981,0.271504,0.128304,0.835936,0.413045,00:23
3,1.389260,0.985951,0.220567,0.087458,0.713560,0.272391,00:24
4,1.103843,0.704417,0.159539,0.070159,0.493929,0.210488,00:25
5,0.861656,0.582303,0.131667,0.051898,0.422454,0.159849,00:24
6,0.629268,0.436759,0.103796,0.033638,0.324792,0.111966,00:25
7,0.477908,0.385466,0.087458,0.030754,0.289432,0.096034,00:25
8,0.400621,0.345902,0.080250,0.024507,0.264984,0.080918,00:24
9,0.349568,0.327834,0.075444,0.020663,0.252921,0.074913,00:22


...

epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.785339,0.523454,0.128304,0.038443,0.398474,0.124980,00:27


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.514056,0.319736,0.076886,0.020663,0.247298,0.072439,00:34
1,0.355777,0.263062,0.060067,0.016819,0.204727,0.058335,00:31
2,0.282375,0.239992,0.054301,0.014416,0.184629,0.055363,00:33
3,0.237279,0.234776,0.045651,0.018741,0.176592,0.058185,00:31
4,0.177753,0.198857,0.040365,0.012494,0.145718,0.053139,00:31
5,0.129067,0.177235,0.036521,0.012494,0.134841,0.042394,00:32
6,0.108689,0.182220,0.038443,0.010091,0.143476,0.038744,00:31
7,0.077910,0.169152,0.032196,0.009130,0.128566,0.040586,00:31
8,0.064028,0.167923,0.034118,0.009611,0.127521,0.040402,00:30
9,0.058904,0.166624,0.033638,0.009611,0.126036,0.040589,00:30


****************************************
***************Size: 224****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.358260,0.317845,0.070159,0.021144,0.250160,0.067686,00:48


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.274271,0.237095,0.049015,0.015377,0.181793,0.055302,00:59
1,0.211532,0.224546,0.042768,0.015377,0.174747,0.049799,00:57
2,0.171042,0.210188,0.040846,0.013455,0.155056,0.055133,00:57
3,0.146246,0.173065,0.034599,0.011533,0.136114,0.036951,00:57
4,0.122981,0.186664,0.036040,0.011533,0.138024,0.048640,00:57
5,0.103979,0.176807,0.031235,0.013936,0.130918,0.045889,00:56
6,0.084865,0.172942,0.026430,0.011533,0.115859,0.057083,00:56
7,0.063056,0.160570,0.027391,0.010572,0.112951,0.047619,00:56
8,0.051598,0.155842,0.024988,0.010091,0.112611,0.043231,00:57
9,0.054379,0.156269,0.025469,0.011052,0.113525,0.042744,00:57


In [17]:
prog_sizing([64, 128, 224], epochs=10)

****************************************
****************Size: 64****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,3.793062,2.720353,0.558866,0.284479,1.738227,0.982126,00:23


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,2.068777,1.252444,0.279193,0.114849,0.883318,0.369126,00:26
1,1.369121,0.881224,0.185488,0.074964,0.615080,0.266144,00:21
2,1.100827,0.690553,0.143681,0.055742,0.487114,0.203439,00:22
3,0.875086,0.519908,0.126382,0.037001,0.394620,0.125288,00:23
4,0.637393,0.415835,0.095147,0.029313,0.297505,0.118331,00:25
5,0.461341,0.352200,0.071120,0.028352,0.250262,0.101938,00:23
6,0.283915,0.256281,0.053820,0.018741,0.189000,0.067281,00:23
7,0.219757,0.203267,0.037482,0.016819,0.146908,0.056360,00:22
8,0.136487,0.180274,0.034118,0.011052,0.129608,0.050666,00:24
9,0.113241,0.178856,0.033157,0.012013,0.127444,0.051412,00:24


****************************************
***************Size: 128****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.816349,0.511677,0.126382,0.037001,0.396984,0.114693,00:25


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.481958,0.272448,0.067275,0.019222,0.215019,0.057429,00:30
1,0.344326,0.284731,0.065834,0.020663,0.208189,0.076542,00:32
2,0.315753,0.325503,0.062951,0.028832,0.221281,0.104223,00:30
3,0.264481,0.258361,0.057184,0.021144,0.186633,0.071728,00:30
4,0.234234,0.201020,0.046612,0.012494,0.151780,0.049240,00:31
5,0.150132,0.159585,0.034118,0.011052,0.123400,0.036184,00:32
6,0.116127,0.149890,0.034599,0.012013,0.111359,0.038531,00:34
7,0.079119,0.140222,0.029793,0.010572,0.101433,0.038789,00:34
8,0.057448,0.132163,0.027871,0.010091,0.095139,0.037024,00:33
9,0.054152,0.131343,0.028832,0.009611,0.095155,0.036188,00:31


****************************************
***************Size: 224****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.340435,0.277488,0.063912,0.024507,0.210076,0.067413,00:47


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.188114,0.183167,0.044690,0.012975,0.145063,0.038104,00:58
1,0.165000,0.204683,0.043729,0.012975,0.154339,0.050344,00:57
2,0.156300,0.193796,0.038443,0.009130,0.151626,0.042170,00:58
3,0.136415,0.162977,0.037963,0.010572,0.124379,0.038598,00:58
4,0.113283,0.177515,0.033638,0.010572,0.133703,0.043811,00:58
5,0.096849,0.149616,0.029313,0.007689,0.116772,0.032844,00:57
6,0.070571,0.154040,0.028352,0.008169,0.112494,0.041547,00:58
7,0.053776,0.141905,0.026430,0.009611,0.106674,0.035231,00:58
8,0.041808,0.136228,0.024988,0.006728,0.103692,0.032536,00:57
9,0.038194,0.136154,0.025469,0.009130,0.102959,0.033195,00:59


In [18]:
prog_sizing([32, 48, 64, 128, 224], epochs=10)

****************************************
****************Size: 32****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,4.158904,3.116308,0.682364,0.311869,2.006430,1.109879,00:24


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,2.682561,1.964522,0.470447,0.196060,1.361388,0.603134,00:23
1,2.095535,1.500143,0.369534,0.140317,1.068777,0.431365,00:22
2,1.698322,1.100544,0.243633,0.103796,0.764703,0.335841,00:25
3,1.327831,0.963063,0.219606,0.080250,0.672531,0.290532,00:23
4,1.086727,0.733237,0.159539,0.060067,0.527358,0.205879,00:25
5,0.850804,0.610335,0.124459,0.052859,0.428840,0.181496,00:25
6,0.661159,0.456756,0.102835,0.040846,0.338507,0.118249,00:22
7,0.475365,0.372957,0.083133,0.033638,0.277773,0.095185,00:21
8,0.387040,0.353154,0.086977,0.028832,0.271396,0.081759,00:21
9,0.337456,0.331709,0.079769,0.029313,0.254720,0.076989,00:25


****************************************
****************Size: 48****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,1.385356,0.960235,0.223450,0.085536,0.680085,0.280150,00:23


...

epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,1.147676,0.776777,0.177319,0.071600,0.565298,0.211479,00:27


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.727765,0.445790,0.103796,0.034118,0.331176,0.114614,00:31
1,0.507812,0.296075,0.070639,0.023066,0.224390,0.071685,00:32
2,0.334937,0.231377,0.054781,0.016819,0.180381,0.050995,00:32
3,0.248394,0.203621,0.047573,0.012494,0.160949,0.042671,00:34
4,0.198701,0.168871,0.037963,0.011533,0.129657,0.039214,00:34
5,0.141330,0.157367,0.035079,0.012013,0.123522,0.033845,00:34
6,0.121630,0.167538,0.037482,0.012975,0.128744,0.038793,00:35
7,0.100249,0.144186,0.029793,0.011052,0.111251,0.032936,00:33
8,0.093092,0.146044,0.033638,0.011052,0.113699,0.032345,00:34
9,0.091164,0.144897,0.033157,0.012013,0.112580,0.032317,00:33


****************************************
***************Size: 224****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.586232,0.473173,0.114368,0.042287,0.350211,0.122962,00:48


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.451138,0.342651,0.082653,0.031235,0.258101,0.084550,00:58
1,0.366523,0.279525,0.063431,0.021144,0.211928,0.067597,00:59
2,0.292428,0.249537,0.059587,0.017299,0.191602,0.057934,00:59
3,0.228513,0.218486,0.048534,0.015377,0.168824,0.049662,00:59
4,0.171156,0.209170,0.048534,0.015377,0.158915,0.050255,00:59
5,0.151807,0.195419,0.045651,0.013455,0.150995,0.044424,00:58
6,0.134379,0.179708,0.041326,0.011052,0.139630,0.040078,00:57
7,0.113777,0.173486,0.038443,0.010572,0.136749,0.036736,00:58
8,0.112900,0.174013,0.038924,0.010091,0.137161,0.036852,00:57
9,0.106706,0.168814,0.039404,0.009611,0.133261,0.035553,00:58


### Using 6 epochs

Maybe we are training too much? Let's train less.

In [19]:
prog_sizing([32, 64, 128, 224], epochs=6)

****************************************
****************Size: 32****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,4.223350,2.986685,0.670831,0.289765,1.981561,1.005124,00:22


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,2.663637,1.973878,0.448342,0.185488,1.357638,0.616240,00:23
1,2.053655,1.461806,0.345026,0.137914,1.017674,0.444132,00:25
2,1.536186,0.979514,0.233061,0.081691,0.710582,0.268932,00:22
3,1.146069,0.736074,0.165786,0.067756,0.529723,0.206352,00:24
4,0.829086,0.531846,0.121576,0.041326,0.392045,0.139801,00:25
5,0.665534,0.498489,0.114849,0.040365,0.367421,0.131068,00:26


****************************************
****************Size: 64****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,1.870233,1.326789,0.324844,0.119173,0.964445,0.362343,00:23


...

epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.972117,0.573909,0.148967,0.042768,0.434885,0.139025,00:30


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.602853,0.350824,0.081211,0.026430,0.268351,0.082473,00:32
1,0.433222,0.256116,0.059106,0.017780,0.190527,0.065589,00:33
2,0.299217,0.225569,0.053340,0.015377,0.171415,0.054154,00:33
3,0.211295,0.194207,0.042768,0.015377,0.139645,0.054562,00:33
4,0.142594,0.164947,0.034118,0.011052,0.120255,0.044691,00:33
5,0.102481,0.165138,0.034118,0.011052,0.120791,0.044347,00:32


****************************************
***************Size: 224****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.493898,0.409541,0.101874,0.025469,0.320781,0.088760,00:47


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.356805,0.281124,0.061028,0.020663,0.213498,0.067626,00:58
1,0.288710,0.225872,0.048534,0.016819,0.173432,0.052440,00:58
2,0.222289,0.185210,0.038924,0.012013,0.141786,0.043424,00:58
3,0.154763,0.177154,0.033157,0.012975,0.133914,0.043241,01:00
4,0.116836,0.169717,0.029313,0.012494,0.127911,0.041806,00:58
5,0.100644,0.163978,0.031235,0.012013,0.123104,0.040874,00:57


In [20]:
prog_sizing([64, 128, 224], epochs=6)

****************************************
****************Size: 64****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,3.762579,2.609209,0.555022,0.254685,1.741313,0.867895,00:20


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,1.982111,1.365971,0.318116,0.123018,0.969191,0.396780,00:22
1,1.339960,0.919171,0.188852,0.095147,0.619797,0.299373,00:26
2,0.905221,0.607074,0.136953,0.046132,0.448390,0.158684,00:26
3,0.572475,0.360821,0.076886,0.027391,0.256134,0.104687,00:27
4,0.357786,0.269746,0.059106,0.020183,0.195866,0.073880,00:25
5,0.241015,0.236025,0.054301,0.017780,0.167045,0.068980,00:24


****************************************
***************Size: 128****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.920515,0.607150,0.152811,0.056223,0.432555,0.174595,00:29


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.550227,0.316887,0.074964,0.026910,0.235051,0.081836,00:33
1,0.453203,0.311757,0.066795,0.024988,0.230032,0.081725,00:32
2,0.358420,0.273717,0.060548,0.021624,0.188667,0.085050,00:32
3,0.239514,0.174924,0.038443,0.013455,0.128691,0.046233,00:32
4,0.138155,0.160606,0.036040,0.009611,0.120322,0.040283,00:30
5,0.096497,0.152782,0.036521,0.008650,0.115605,0.037177,00:31


****************************************
***************Size: 224****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.393244,0.353334,0.085055,0.024507,0.280665,0.072669,00:48


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.290929,0.220637,0.049976,0.017299,0.166026,0.054611,00:59
1,0.249458,0.264452,0.054781,0.019702,0.190018,0.074434,00:59
2,0.185101,0.221444,0.047573,0.017780,0.161943,0.059501,00:58
3,0.132881,0.162844,0.035079,0.013455,0.123501,0.039343,00:58
4,0.095318,0.148586,0.030274,0.010091,0.113107,0.035479,00:59
5,0.070303,0.147172,0.028832,0.009130,0.112608,0.034564,01:00


In [21]:
prog_sizing([32, 48, 64, 128, 224], epochs=6)

****************************************
****************Size: 32****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,4.164460,3.026726,0.663623,0.306583,1.986262,1.040465,00:25


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,2.748651,1.968233,0.471889,0.189332,1.385521,0.582712,00:24
1,2.061898,1.442669,0.335416,0.143681,1.017890,0.424780,00:23
2,1.519818,1.081555,0.230178,0.110043,0.708949,0.372607,00:22
3,1.136953,0.704397,0.164344,0.070639,0.502151,0.202245,00:24
4,0.795527,0.533952,0.130226,0.043248,0.403681,0.130270,00:21
5,0.637507,0.499829,0.124459,0.039404,0.376684,0.123145,00:22


****************************************
****************Size: 48****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,1.617146,1.182365,0.277271,0.113888,0.856031,0.326334,00:20


...

epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,1.152820,0.771685,0.187890,0.065834,0.585688,0.185996,00:26


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.744372,0.438524,0.104757,0.036040,0.330464,0.108060,00:29
1,0.489161,0.293860,0.068236,0.022585,0.227072,0.066787,00:30
2,0.350339,0.247377,0.057665,0.019222,0.189369,0.058008,00:30
3,0.246128,0.183397,0.046132,0.012975,0.138841,0.044556,00:30
4,0.180161,0.173767,0.039404,0.011533,0.132008,0.041760,00:30
5,0.158222,0.172995,0.042768,0.012494,0.131161,0.041834,00:31


****************************************
***************Size: 224****************
****************************************


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.708663,0.521784,0.135031,0.041326,0.406819,0.114966,00:46


epoch,train_loss,valid_loss,disease_err,variety_err,disease_loss,variety_loss,time
0,0.528055,0.398089,0.098991,0.036040,0.303108,0.094981,00:56
1,0.418745,0.327078,0.079289,0.030274,0.240093,0.086985,00:56
2,0.305685,0.237273,0.059106,0.020663,0.179425,0.057848,00:56
3,0.244925,0.226729,0.055742,0.018741,0.173865,0.052864,00:57
4,0.202977,0.207105,0.048054,0.017299,0.156921,0.050184,00:57
5,0.184796,0.206490,0.050937,0.017299,0.156276,0.050214,00:56
